# D7041E Miniproject

Group ID: MINI-PROJECT 14

Ahmad Allahham
[ahmall-0@student.ltu.se] | 940120-0556 |

Arian Asghari
[ariasg-0@student.ltu.se] | 010721-7051 |

Hannes Furhoff
[hanfur-0@student.ltu.se] | 010929-4710 |

## Grade requirements
### G3: Run and understand a publicly available model on a one selected dataset.
We have chosen to work with the MLP model (regressive).

### G3: Choose a dataset.
Our data set is a collection of synthetic housing data, with various parameters (eg. rooms, year built) and price.

### G3: Implement tutorial.
NOT DONE YET

### G3: Test performance for different configurations of the perceptron.
NOT DONE YET

### G3: Document the performance.
NOT DONE YET

### G4: You should use data pre-processing
For preprocessing a few steps were done:
- Converting all values to float type.
- Encoding the categorical "neighborhood" column to one-hot and then to float.

### G4: Systematically choose the hyper-parameters of the model
NOT DONE YET

### G4: Use cross-validation for training 
NOT DONE YET

### G4: Use different seeds and  recorded performance statistics with various performance metrics
NOT DONE YET


## Additional info
Github link: `https://github.com/afy/d7041e_miniproject` <br/>
Dataset link: `https://www.kaggle.com/datasets/muhammadbinimran/housing-price-prediction-data` <br/>
Based on tutorial: `https://machinelearningmastery.com/building-a-regression-model-in-pytorch/` <br/>
Youtube demo link: `NOT DONE YET` <br/>

## Code

In [157]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
import itertools
from sklearn.preprocessing import OneHotEncoder

In [158]:
# Misc

# Returns from run
class ModelMetrics:
    def __init__(self, _mse, _rmse, _mae, _time, _settings):
        self.mse = _mse
        self.rmse = _rmse
        self.mae = _mae
        self.training_time = _time
        self.run_settings = _settings # The settings the model was run on

# Settings given to the training function
class ModelSettings:
    def __init__(self, _layers, _epochs, _batch):
        self.layers = _layers
        self.epochs = _epochs
        self.batch_size = _batch
        self.optimizer_learning_rate = 0.001

In [159]:
# Preprocessing 

# Read data
raw_data = np.loadtxt("housing_data.csv", delimiter=",", dtype=str)
column_names = raw_data[0]
raw_data = raw_data[1:]

# For "Neighborhood" column:
# Str -> OH-encoding -> str repr -> float repr 
# Eg. "Urban" -> [0 1 0] -> "010" -> (0)10.0
d = raw_data[:,3].reshape(-1, 1)
oh_enc = OneHotEncoder(dtype=int)
oh_enc.fit(d)
oh_d = oh_enc.transform(d).toarray().astype(str)
for r in range(len(oh_d)):
    tf = int(''.join(oh_d[r]))
    raw_data[r,3] = tf

# Type convert data
raw_data = raw_data.astype(float)
housing_data, housing_prices = raw_data[:,0:5], raw_data[:,5]

In [160]:
# Model training

def evaluateModel(settings: ModelSettings, data_train: np.ndarray, data_test: np.ndarray) -> ModelMetrics:
    seq =  nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3),
      nn.ReLU(),
      nn.Flatten(),
      nn.Linear(26 * 26 * 10, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 10)
    )

    optimizer = optim.Adam(seq.parameters(), lr = settings.optimizer_learning_rate) 

    


In [161]:
# Cross-validation

def k_fold_validation(k: int, data: np.ndarray, prices: np.ndarray, settings: ModelSettings) -> ModelMetrics:
    return evaluateModel(settings, 0, 0)

In [164]:
# Run

# Running params
# The idea is to cycle through the permutations of these
k_folds = 3                             # For cross-fold validation
layer_length = [1, 3]                   # 1 'layer' = 1 neuron + activation in sequence
in_features_sizes = [1, 4, 8, 12, 16, 24, 48, 64, 128]            # Sizes of linear features per-layer
out_features_sizes = [1, 4, 8, 12, 16, 24, 48, 64, 128]           # - || -

# Other params
max_saved_runs = 10

# Setup
possible_layers = [i for i in range(layer_length[0], layer_length[1] + 1)]
best_runs = np.ndarray(max_saved_runs).astype(object)

# Train and test model for all argument combinations
# Save the {max_saved_runs} best runs
combos = itertools.product(possible_layers, in_features_sizes, out_features_sizes)

for argcomb in combos:
    l_order = []
    for li in range(argcomb[0]):
        l_order.append(nn.Linear(1, 1))
        l_order.append(nn.ReLU)

    settings = ModelSettings(
        l_order,
        100,
        10
    )
    m = k_fold_validation(k_folds, housing_data, housing_prices, settings)
    break

<generator object Module.parameters at 0x000001D4D520C3C0>


In [163]:
for r in range(best_runs.size):
    print(best_runs[r].rmse)

AttributeError: 'float' object has no attribute 'rmse'